In [1]:
# パッケージのimport
from math import sqrt
from itertools import product

import torch
from torch.autograd import Function
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import pandas as pd

In [2]:
#34層にわたる，vgg-module
def make_vgg():
    layers = []
    in_channels = 3 #color channels
    
    #vgg-moduleで使用する畳み込み層やマックスプーリングのチャネル数
    cfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'MC', 512, 512, 512, 'M', 512, 512, 512]
    
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        elif v=='MC':
            #ceilは出力サイズを，計算結果(float)に対して，切り上げて整数にするモード
            #デフォルでは出力サイズを計算結果(float)に対して，切り下げで整数にするfloor-mode
            layers+=[nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels=v
            
    pool5 = nn.MaxPool2d(kernel_size=3,stride=1,padding=1)
    conv6 = nn.Conv2d(512,1024,kernel_size=3,padding=6,dilation=6)
    conv7 = nn.Conv2d(1024,1024,kernel_size=1)
    layers+=[pool5, conv6, nn.ReLU(inplace=True), conv7, nn.ReLU(inplace=True)]
    return nn.ModuleList(layers)

#動作確認
vgg_test = make_vgg()
print(vgg_test)

ModuleList(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (17): Conv2d(256, 512, kernel_siz

In [3]:
#8層にわたる，extras-module
def make_extras():
    layers = []
    in_channels = 1024 #vgg-moduleから出力された，extraに入力される画像
    
    #extraモジュールの畳み込み層のチャネル数を設定するコンフィギュレーション
    cfg = [256, 512, 128, 256, 128, 256, 128, 256]
    
    layers += [nn.Conv2d(in_channels, cfg[0], kernel_size=(1))]
    layers += [nn.Conv2d(cfg[0], cfg[1], kernel_size=(3), stride=2, padding=1)]
    layers += [nn.Conv2d(cfg[1], cfg[2], kernel_size=(1))]
    layers += [nn.Conv2d(cfg[2], cfg[3], kernel_size=(3), stride=2, padding=1)]
    layers += [nn.Conv2d(cfg[3], cfg[4], kernel_size=(1))]
    layers += [nn.Conv2d(cfg[4], cfg[5], kernel_size=(3))]
    layers += [nn.Conv2d(cfg[5], cfg[6], kernel_size=(1))]
    layers += [nn.Conv2d(cfg[6], cfg[7], kernel_size=(3))]
    
    return nn.ModuleList(layers)

#動作確認
extras_test = make_extras()
print(extras_test)

ModuleList(
  (0): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
  (1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (2): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1))
  (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (4): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (6): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
)


In [4]:
#デフォルトボックスのオフセットを出力するloc_layers
#デフォルトボックスに対する各クラスの信頼度confidenceを出力するconf_layersを作成

def make_loc_conf(num_classes=21, bbox_aspect_num=[4,6,6,6,4,4]):
    
    loc_layers = []
    conf_layers = []
    
    #VGGの22層目，conv4_3(source1)に対する畳み込み層
    loc_layers += [nn.Conv2d(512, bbox_aspect_num[0] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(512, bbox_aspect_num[0]*num_classes, kernel_size=3, padding=1)]
    
    #VGGの最終層(source2)に対する畳み込み層
    loc_layers += [nn.Conv2d(1024, bbox_aspect_num[1] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(1024, bbox_aspect_num[1]*num_classes, kernel_size=3, padding=1)]
    
    #extraの(source3)に対する畳み込み層
    loc_layers += [nn.Conv2d(512, bbox_aspect_num[2] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(512, bbox_aspect_num[2]*num_classes, kernel_size=3, padding=1)]

    #extraの(source4)に対する畳み込み層
    loc_layers += [nn.Conv2d(256, bbox_aspect_num[3] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(256, bbox_aspect_num[3]*num_classes, kernel_size=3, padding=1)]
    
    #extraの(source5)に対する畳み込み層
    loc_layers += [nn.Conv2d(256, bbox_aspect_num[4] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(256, bbox_aspect_num[4]*num_classes, kernel_size=3, padding=1)]
    
    #extraの(source6)に対する畳み込み層
    loc_layers += [nn.Conv2d(256, bbox_aspect_num[5] * 4, kernel_size=3, padding=1)]
    conf_layers += [nn.Conv2d(256, bbox_aspect_num[5]*num_classes, kernel_size=3, padding=1)]

    return nn.ModuleList(loc_layers), nn.ModuleList(conf_layers)

#動作確認
loc_test, conf_test = make_loc_conf()
print(loc_test)
print(conf_test)

ModuleList(
  (0): Conv2d(512, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)
ModuleList(
  (0): Conv2d(512, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)


In [5]:
#conv4_3からの出力をscale=20のL2Normで正規化する層
class L2Norm(nn.Module):
    def __init__(self, input_channels=512, scale=20):
        super(L2Norm, self).__init__() #親クラスのコンストラクタ実行
        self.weight = nn.Parameter(torch.Tensor(input_channels))
        self.scale = scale #係数weightの初期値として設定する値
        self.reset_parameters() #パラメータの初期化
        self.eps = 1e-10
    
    def reset_parameters(self):
        #結合パラメタを大きさscaleの値にする初期化
        init.constant_(self.weight, self.scale) #weightの値が全てscaleになる
        
    def forward(self, x):
        '''38*38の特徴量に対して，512チャネルに渡って2乗和野ルートを求めた38*38個の値を使用し，各特徴量を正規化してから係数を掛け算する層'''
        
        #各チャネルにおける38*38個の特徴量のチャネル方向の2乗和を計算し，更にルートを求め，割り算をして正規化
        #normのテンソルサイズはtorch.Size([batch_num, 1, 38, 38])
        norm = x.pow(2).sum(dim=1, keepdim=True).sqrt()+self.eps
        x=torch.div(x,norm)
        
        #係数をかける．係数はチャネルごとに1つで，512個の係数を持つ
        #self.weightのテンソルサイズはtorch.Size([512])なので，torch.Size([batch_num, 512, 38, 38])にする
        #unsqueeze(dim)でdimの場所に次元を追加する
        weights = self.weight.unsqueeze(0).unsqueeze(2).unsqueeze(3).expand_as(x)
        out = weights * x
        
        return out

In [6]:
#デフォルトボックスを出力するクラス
class DBox(object):
    def __init__(self, cfg):
        super(DBox, self).__init__()
        
        #初期設定
        self.image_size=cfg['input_size'] #画像のサイズ300
        #[38, 19, ...]各sourceの特徴量マップのサイズ
        self.feature_maps=cfg['feature_maps']
        self.num_priors=len(cfg["feature_maps"]) #sourceの数=6
        self.steps=cfg['steps'] #[8,16,...]DBoxのピクセルサイズ
        self.min_sizes=cfg['min_sizes'] #[30,60,...]小さい正方形のDBoxのピクセルサイズ
        self.max_sizes=cfg['max_sizes'] #[60,111,...]大きい正方形のDBoxのピクセルサイズ
        self.aspect_ratios=cfg['aspect_ratios'] #長方形のDBoxのアスペクト比
        
    def make_dbox_list(self):
        #Dboxを作成する
        mean=[]
        #feature_maps = [38, 19, 10, 5, 3, 1]
        for k, f in enumerate(self.feature_maps):
            for i, j in product(range(f), repeat=2): #fまでの数で2ペアの組み合わせを作るf_P_2個
                #特徴量の画像サイズ
                #300 / 'steps' : [8,16,32,64,100,300]
                f_k = self.image_size / self.steps[k]
                
                #DBoxの中心座標x,y　ただし，0~1で規格化している
                cx = (j+0.5)/f_k
                cy = (i+0.5)/f_k
                
                #アスペクト比1の小さいDBox[cx,cy,width,height]
                #'min_sizes' : [30, 60, 111, 162, 213, 264]
                s_k = self.min_sizes[k]/self.image_size
                mean += [cx, cy, s_k ,s_k]
                
                #アスペクト比1の大きいDBox[cx, cy, width, height]
                #'max_sizes' : [60, 111, 162, 213, 264, 315]
                s_k_prime = sqrt(s_k* (self.max_sizes[k]/self.image_size))
                mean += [cx, cy, s_k_prime, s_k_prime]
                
                #その他のアスペクト比のdefBox[cx, cy, width, height]
                for ar in self.aspect_ratios[k]:
                    mean += [cx, cy, s_k*sqrt(ar), s_k/sqrt(ar)]
                    mean += [cx, cy, s_k/sqrt(ar), s_k*sqrt(ar)]
                    
        #DBoxをテンソルに変換 torch.Size([8732, 4])
        output = torch.Tensor(mean).view(-1,4)
        
        #DBoxが画像の外にはみ出るのを防ぐため，大きさを最小0,最大1にする
        output.clamp_(max=1, min=0)
        
        return output

In [7]:
#動作の確認
#SSD300の設定
ssd_cfg = {
    'num_classes':21, #背景クラスを含めた合計クラス
    'input_size':300, #画像の入力サイズ
    'bbox_aspect_num':[4,6,6,6,4,4], #出力するDBoxのアスペクト比
    'feature_maps':[38, 19, 10, 5, 3, 1], #各sourceの画像サイズ
    'steps':[8, 16, 32, 64, 100, 300], #DBOXの大きさを決める
    'min_sizes':[30, 60, 111, 162, 213, 264], #DBOXの大きさを決める
    'max_sizes':[60, 111, 162, 213, 264, 315], #DBOXの大きさを決める
    'aspect_ratios':[[2], [2,3], [2,3], [2,3], [2], [2]],
}

#DBOX作成
dbox = DBox(ssd_cfg)
dbox_list = dbox.make_dbox_list()

#DBoxの出力を確認する
pd.DataFrame(dbox_list.numpy())

,0,1,2,3
0,0.013333,0.013333,0.100000,0.100000
1,0.013333,0.013333,0.141421,0.141421
2,0.013333,0.013333,0.141421,0.070711
3,0.013333,0.013333,0.070711,0.141421
4,0.040000,0.013333,0.100000,0.100000
...,...,...,...,...
8727,0.833333,0.833333,0.502046,1.000000
8728,0.500000,0.500000,0.880000,0.880000
8729,0.500000,0.500000,0.961249,0.961249
8730,0.500000,0.500000,1.000000,0.622254


In [9]:
#SSDクラスを作成する
class SSD(nn.Module):
    
    def __init__(self,phase,cfg):
        super(SSD, self).__init__()
        
        self.phase = phase #train or inference
        self.num_classs = cfg["num_classes"] #クラス数=21
        
        #SSDのネットワークを作る
        self.vgg = make_vgg()
        self.extras = make_extras()
        self.L2Norm = L2Norm()
        self.loc, self.conf = make_loc_conf(cfg['num_classes'], cfg['bbox_aspect_num'])
        
        #DBoxを作成
        dbox = DBox(cfg)
        self.dbox_list = dbox.make_dbox_list()
        
        #推論時はクラス[Detect]を用意
        if phase == 'inference':
            self.detect = Detect()
            
#動作確認
ssd_test = SSD(phase='train', cfg=ssd_cfg)
print(ssd_test)  

SSD(
  (vgg): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, cei

In [10]:
#オフセット情報を使い，DBoxをBBoxに変換する関数
def decode(loc, dbox_list):
    """
    オフセット情報を使い，DBoxをBBoxに変換する
    
    parameters
    loc : [8732, 4]
        SSDモデルで推論するオフセット情報
    dbox_list : [8732, 4]
        DBoxの情報
        
    returns
    boxes : [xmin, ymin, xmax, ymax]
        BBoxの情報
    """
    
    #DBoxは[cx, cy, width, height]で格納されている
    #locも[⊿cx, ⊿cy, ⊿width, ⊿height]で格納されている
    
    #オフセット情報からBBoxを求める
    boxes = torch.cat((dbox_list[:,:2]+loc[:,:2]*0.1*dbox_list[:,2:],
                       dbox_list[:,2:] * torch.exp(loc[:,2:] * 0.2)), dim=1) #cat:concatenate:dimに対して結合する関数
    #dboxesのサイズはtorch.Size([8732,4])になる
    
    #BBoxの座標情報を[cx,cy,width,height]から[xmin,ymin,xmax,ymax]に
    boxes[:,:2]-=boxes[:,2:]/2
    boxes[:,2:]+=boxes[:,:2]
    
    return boxes

#Non-Maximum Suppressionを行う関数
def nm_suppression(boxes, scores, overlap=0.45, top_k=200):
    """
    Non-Maximum Suppression
    boxesのうちかぶりすぎ(overlap以上)のbboxを削除する
    
    parameters
    boxes : [確信度閾値(0.01)を超えたBBox数,4] BBox情報
    scores : [確信度閾値(0.01)を超えたBBox数,4] confの情報
    
    returns
    keep : リスト confの降順にnmsを通過したindexが格納
    conut : int nmsを通過したBBoxの数
    """
    
    #return の雛形
    count = 0
    keep = scores.new(scores.size(0)).zero_().long() #keep:torch.Size([確信度閾値を超えたBBox数]), 要素は全部0
    
    #各BBoxの面積areaを計算
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    area = torch.mul(x2 - x1, y2 - y1)
    
    #boxesをコピー．あとでBBoxの被り度合いIOUの計算に使用する際の雛形として用意
    tmp_x1 = boxes.new()
    tmp_x2 = boxes.new()
    tmp_y1 = boxes.new()
    tmp_y2 = boxes.new()
    tmp_w = boxes.new()
    tmp_h = boxes.new()
    
    #scoresを昇順に
    v, idx = scores.sort(0) #values, indicates
    
    #idxの要素数が0出ない限りループ
    while idx.numel() > 0:
        i = idx[-1] #現在のconf最大のindexをiに
        
        #keepの現在の最後にconf最大のindexを格納する
        #このindexのBBoxと被りの大きいBBoxをこれから消去する
        keep[conut] = i
        count += 1
        
        #最後のBBoxになった場合はループを抜ける
        if idx.size(0) == 1:
            break
            
        #現在のconf最大のindexをkeepに格納したので，idxを1つ減らす
        idx = idx[:-1]
        
        #これからkeepに格納したBBoxと被りの大きいBBoxを抽出して除去する
        #1つ減らしたidxまでのBBoxを，outに指定した変数として作成する
        torch.index_select(x1, 0, idx, out=tmp_x1) #input=x1のうち，dim=0で示される次元において，idxで示される場所のみ取り出し，outに格納する
        torch.index_select(y1, 0, idx, out=tmp_y1)
        torch.index_select(x2, 0, idx, out=tmp_x2)
        torch.index_select(y2, 0, idx, out=tmp_y2)
        
        #すべてのBBoxに対して，現在のBBox=indexがiと被っている値までに設定(clamp)
        tmp_x1 = torch.clamp(tmp_x1, min=x1[i])
        tmp_y1 = torch.clamp(tmp_y1, min=y1[i])
        tmp_x2 = torch.clamp(tmp_x2, max=x2[i])
        tmp_y2 = torch.clamp(tmp_y2, max=y2[i])
        
        #wとhのテンソルサイズをindexを1つ減らしたものにする
        tmp_w.resize_as_(tmp_x2)
        tmp_h.resize_as_(tmp_y2)
        
        #clampした状態でのBBoxの幅と高さを求める
        tmp_w = tmp_x2 - tmp_x1
        tmp_h = tmp_y2 - tmp_y1
        
        #幅や高さが負になっているものは0にする
        tmp_w = torch.clamp(tmp_w, min=0.0)
        tmp_h = torch.clamp(tmp_h, min=0.0)
        
        #clampされた状態での被っている面積を求める
        inter = tmp_w * tmp_h
        
        #IoU = intersect部分 / (area(a)+area(b)-intersect部分)の計算
        rem_areas = torch.index_select(area, 0, idx) #各BBoxの元の面積
        union = (rem_areas - inter) + area[i] #2つのエリアのORの面積
        IoU = inter / union
        
        #IoUがoverlapより小さいidxのみ残す
        idx = idx[IoU.le(overlap)] #leはLess than or equal to の処理をする演算
        
    #while-loopを抜けたら終了
    return keep, count

In [12]:
#SSDの推論時にconfとlocの出力から，被りを消去したBBoxを出力する

class Detect(Function):
    
    def __init__(self, conf_thresh=0.01, top_k=200, nms_thresh=0.45):
        self.softmax=nn.Softmax(dim=-1) #confをソフトマックス関数で正規化するために用意
        self.conf_thresh=conf_thresh #confがconf_thresh=0.01より高いDBoxのみを扱う
        self.top_k=top_k #nm_supressionでconfの高いtop_k個を計算にしようする.top_k=200
        self.nms_thresh=nms_thresh #nm_suppressionでIoUがnms_threshより大きいと同一物体へのBBoxとみなす
        
    def forward(self, loc_data, conf_data, dbox_list):
        """
        順伝搬の計算を実行
        
        parameters
        loc_data:[batch_num, 8732, 4] オフセット情報
        conf_data:[batch_num, 8732, num_classes] 検出の確信度
        dbox_list:[8732,4] DBoxの情報
        
        returns
        output:torch.Size([batch_num,21,200,5])
            (batch_num, クラス, confのtop200, BBoxの情報)
        """
        
        #各サイズを取得
        num_batch = loc_data.size(0) #ミニバッチのサイズ
        num_dbox = loc_data.size(1) #DBoxの数=8732
        num_classes = conf_data.size(2) #クラスの数=21
        
        #confはソフトマックスを適用して正規化する
        conf_data = self.softmax(conf_data)
        
        #出力の型を生成する．テンソルサイズは[minibatch数，21, 200, 5]
        output = torch.zeros(num_batch, num_classes, self.top_k, 5)
        
        #conf_dataを[batch_num, 8732, num_classes]から[batch_num, num_classes, 8732]に順番変更
        conf_preds = conf_data.transpose(2,1)
        
        #ミニバッチごとのループ
        for i in range(num_batch):
            #1. locとDBoxから修正したBBox[xmin, ymin, xmax, ymax]を求める
            decoded_boxes = decode(loc_data[i], dbox_list)
            
            #confのコピー生成
            conf_scores = conf_preds[i].clone()
            
            #画像クラスごとのループ(背景クラスのindexである0は計算せず，index=1から)
            for cl in range(1, num_classes):
                
                #2.confの閾値を超えたBBoxを取り出す．
                #confの閾値を超えているかのマスクを作成し，閾値を超えたconfのインデックスをc_maskとして作成
                c_mask = conf_scores[cl].gt(self.conf_thresh)
                #gtはGreater thanのこと．gtにより閾値を超えたものが1，以下が0になる
                #conf_scores:torch.Size([21, 8732]), c_mask:torch.Size([8732])
                
                #scoresはtorch.Size([閾値を超えたBBox数])
                scores = conf_scores[cl][c_mask]
                
                #閾値を超えたconfがない場合，何もしない
                if scores.nelement() == 0: #nelementで要素数を求められる
                    continue
                    
                #c_maskを，decoded_boxesに適用できるようにサイズを変更
                l_mask = c_mask.unsqueeze(1).expand_as(decoded_boxes)
                #location_mask:torch.Size([8732, 4])
                
                boxes = decoded_boxes[l_mask].view(-1,4)
                #decoded_boxes[l_mask]で1次元になてしまうのでviewで(閾値を超えた数, 4)にリサイズする
                
                #3. Non-Maximum Suppression
                ids, count = nm_suppression(boxes, scores,self.nms_thresh, self.top_k)
                #ids:confの降順にnon-maximum suppressionを通過したindexが格納
                #count:non-maximum suppressionを通過したbboxの数
                
                #outputにNon-Maximum Suppressionを抜けた結果を格納
                output[i, cl, :count] = torch.cat((scores[ids[:count]].unsqueeze(1),boxes[ids[:count]]), 1)
                
        return output #torch.Size([1,21,200,5])

In [ ]:
#SSDクラスを作成する
class SSD(nn.Module):
    
    def __init__(self,phase,cfg):
        super(SSD, self).__init__()
        
        self.phase = phase #train or inference
        self.num_classs = cfg["num_classes"] #クラス数=21
        
        #SSDのネットワークを作る
        self.vgg = make_vgg()
        self.extras = make_extras()
        self.L2Norm = L2Norm()
        self.loc, self.conf = make_loc_conf(cfg['num_classes'], cfg['bbox_aspect_num'])
        
        #DBoxを作成
        dbox = DBox(cfg)
        self.dbox_list = dbox.make_dbox_list()
        
        #推論時はクラス[Detect]を用意
        if phase == 'inference':
            self.detect = Detect()
            
    def forward(self, x):
        sources = list() #locとconfへの入力source1~6
        loc = list() #locの出力を格納
        conf = list() #confの出力を格納
        
        #vggのconv4_3まで計算
        for k in range(23):
            x=self.vgg[k](x)
            
        #conv4_3の出力をL2Normに入力し，source1を作成，sourcesに追加
        source1 = self.L2Norm(x)
        sources.append(source1)
        
        #vggを最後まで計算し，source2を作成，sourcesに追加
        for k in range(23, len(self.vgg)):
            x=self.vgg[k](x)
            
        sources.append(x)
        
        #extrasのconvとReLUを計算，source3~6をsourcesに追加
        for k, v in enumertate(self.extras):
            x = F.relu(v(x), inplace=True)
            if k%2==1: #conv→ReLU→conv→ReLUをしたらsourcesに追加
                sources.append(x)
        
        #source1~6にそれぞれ対応する畳み込みを1回ずつ適用する
        #zipでforループの複数リストの要素を取得
        #source1~6まであるので6回ループが回る
        for (x,l,c) in zip(sources, self.loc, self.conf):
            #permuteは要素の順番を入れ替え
            loc.append(l(x).permute(0,2,3,1).contiguous())
            conf.append(c(x).permute(0,2,3,1).contiguous())
            """
            l(x)とc(x)で畳み込み実行
            l(x)とc(x)の出力サイズは[batch_num, 4*アスペクト比の種類数, featuremapの高さ, featuremap幅]
            sourceによって，アスペクト比の種類数が異なり，面倒なので順番を入れかえて整える
            permuteで要素の順番を入れ替え，[minibatch数，featuremap数，featuremap数，4*アスペクト比の種類数]へ
            torch.contiguous()はメモリ上で要素を連続的配置し直す命令
            あとでview関数を使用．このview関数を行うためには対象の変数がメモリ上で連続的に配置されている必要がある．
            """
            
        #さらにlocとconfの形を変形．loc.Size([batch_num, 34928]), conf.Size([batch_num, 183372])
        loc = torch.cat([o.view(o.size(0), -1) for o in loc], 1)
        conf = torch.cat([o.view(o.size(0), -1) for o in conf])
        
        #さらにlocとconfの形を変える
        #loc.Size([batch_num, 8732, 4]), conf.Size([batch_num, 8732, 21])
        loc = loc.view(loc.size(0), -1, 4)
        conf = conf.view(conf.size(0), -1, self.num_classs)
        
        #最後に出力する
        output = (loc, conf, self.dbox_list)
        
        if self.phase == "infernece":
            #クラス[Detect]のforwardを実行．返り値のサイズはtoch.Size([batch_num, 21, 200, 5])
            return self.detect(output[0], output[1], output[2])
        else: #学習時
            return output #返り値は(loc, conf, dbox_list)のタプル    